## Demonstration of the topic coherence pipeline in Gensim

### Introduction

We will be using the `u_mass` and `c_v` coherence for two different LDA models: a "good" and a "bad" LDA model. The good LDA model will be trained over 50 iterations and the bad one for 1 iteration. Hence in theory, the good LDA model will be able come up with better or more human-understandable topics. Therefore the coherence measure output for the good LDA model should be more (better) than that for the bad LDA model. This is because, simply, the good LDA model usually comes up with better topics that are more human interpretable.

In [1]:
import numpy as np
import logging
try:
    import pyLDAvis.gensim
except ImportError:
    ValueError("SKIP: please install pyLDAvis")
    
import json
import warnings
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity

from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.models.hdpmodel import HdpModel
from gensim.models.wrappers import LdaVowpalWabbit, LdaMallet
from gensim.corpora.dictionary import Dictionary
from numpy import array

/home/ssreehari10/anaconda2/envs/gensim2/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/ssreehari10/anaconda2/envs/gensim2/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/ssreehari10/anaconda2/envs/gensim2/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/ssreehari10/anaconda2/envs/gensim2/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/ssreehari10/anaconda2/envs/gensim2/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is dep

### Set up logging

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


### Set up corpus

As stated in table 2 from [this](http://www.cs.bham.ac.uk/~pxt/IDA/lsa_ind.pdf) paper, this corpus essentially has two classes of documents. First five are about human-computer interaction and the other four are about graphs. We will be setting up two LDA models. One with 50 iterations of training and the other with just 1. Hence the one with 50 iterations ("better" model) should be able to capture this underlying pattern of the corpus better than the "bad" LDA model. Therefore, in theory, our topic coherence for the good LDA model should be greater than the one for the bad LDA model.

In [3]:
texts = [['human', 'interface', 'computer'],
         ['survey', 'user', 'computer', 'system', 'response', 'time'],
         ['eps', 'user', 'interface', 'system'],
         ['system', 'human', 'system', 'eps'],
         ['user', 'response', 'time'],
         ['trees'],
         ['graph', 'trees'],
         ['graph', 'minors', 'trees'],
         ['graph', 'minors', 'survey']]

In [4]:
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(12 unique tokens: ['trees', 'system', 'response', 'computer', 'graph']...) from 9 documents (total 29 corpus positions)


### Set up two topic models

We'll be setting up two different LDA Topic models. A good one and bad one. To build a "good" topic model, we'll simply train it using more iterations than the bad one. Therefore the `u_mass` coherence should in theory be better for the good model than the bad one since it would be producing more "human-interpretable" topics.

In [5]:
goodLdaModel = LdaModel(corpus=corpus, id2word=dictionary, iterations=50, num_topics=2)
badLdaModel = LdaModel(corpus=corpus, id2word=dictionary, iterations=1, num_topics=2)

INFO:gensim.models.ldamodel:using symmetric alpha at 0.5
INFO:gensim.models.ldamodel:using symmetric eta at 0.5
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online LDA training, 2 topics, 1 passes over the supplied corpus of 9 documents, updating model once every 9 documents, evaluating perplexity every 9 documents, iterating 50x with a convergence threshold of 0.001000
DEBUG:gensim.models.ldamodel:bound: at document #0
INFO:gensim.models.ldamodel:-3.599 per-word bound, 12.1 perplexity estimate based on a held-out corpus of 9 documents with 29 words
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #9/9
DEBUG:gensim.models.ldamodel:performing inference on a chunk of 9 documents
DEBUG:gensim.models.ldamodel:6/9 documents converged within 50 iterations
DEBUG:gensim.models.ldamodel:updating topics
INFO:gensim.models.ldamodel:topic #0 (0.500): 0.146*system + 0.117*graph + 0.116*trees + 0.103*minors + 0.089*user + 0.087*eps + 

### Using U_Mass Coherence

In [6]:
goodcm = CoherenceModel(model=goodLdaModel, corpus=corpus, dictionary=dictionary, coherence='u_mass')

In [7]:
badcm = CoherenceModel(model=badLdaModel, corpus=corpus, dictionary=dictionary, coherence='u_mass')

### View the pipeline parameters for one coherence model

Following are the pipeline parameters for `u_mass` coherence. By pipeline parameters, we mean the functions being used to calculate segmentation, probability estimation, confirmation measure and aggregation as shown in figure 1 in [this](http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf) paper.

In [8]:
print(goodcm)

Coherence_Measure(seg=<function s_one_pre at 0x7f90272c4620>, prob=<function p_boolean_document at 0x7f90272c4b70>, conf=<function log_conditional_probability at 0x7f90272c4c80>, aggr=<function arithmetic_mean at 0x7f90272c4f28>)


### Interpreting the topics

As we will see below using LDA visualization, the better model comes up with two topics composed of the following words:
1. goodLdaModel:
    - __Topic 1__: More weightage assigned to words such as "system", "user", "eps", "interface" etc which captures the first set of documents.
    - __Topic 2__: More weightage assigned to words such as "graph", "trees", "survey" which captures the topic in the second set of documents.
2. badLdaModel:
    - __Topic 1__: More weightage assigned to words such as "system", "user", "trees", "graph" which doesn't make the topic clear enough.
    - __Topic 2__: More weightage assigned to words such as "system", "trees", "graph", "user" which is similar to the first topic. Hence both topics are not human-interpretable.

Therefore, the topic coherence for the goodLdaModel should be greater for this than the badLdaModel since the topics it comes up with are more human-interpretable. We will see this using `u_mass` and `c_v` topic coherence measures.

### Visualize topic models

In [9]:
pyLDAvis.enable_notebook()

In [10]:
pyLDAvis.gensim.prepare(goodLdaModel, corpus, dictionary)

DEBUG:gensim.models.ldamodel:performing inference on a chunk of 9 documents
DEBUG:gensim.models.ldamodel:9/9 documents converged within 50 iterations


PreparedData(topic_coordinates=            Freq  cluster  topics         x    y
topic                                           
0      50.900005        1       1  0.015493  0.0
1      49.099995        1       2 -0.015493  0.0, topic_info=     Category      Freq       Term     Total  loglift  logprob
term                                                          
11    Default  2.000000       time  2.000000  12.0000  12.0000
7     Default  2.000000     minors  2.000000  11.0000  11.0000
2     Default  2.000000   response  2.000000  10.0000  10.0000
3     Default  2.000000   computer  2.000000   9.0000   9.0000
10    Default  2.000000  interface  2.000000   8.0000   8.0000
1     Default  3.000000     system  3.000000   7.0000   7.0000
4     Default  2.000000      graph  2.000000   6.0000   6.0000
0     Default  2.000000      trees  2.000000   5.0000   5.0000
6     Default  2.000000        eps  2.000000   4.0000   4.0000
9     Default  2.000000       user  2.000000   3.0000   3.0000
5     Default  2.000000     survey  2.000000   2.0000   2.0000
8     Default  2.000000      human  2.000000   1.0000   1.0000
7      Topic1  1.515831     minors  2.129507   0.3354  -2.2760
4      Topic1  1.731552      graph  2.834322   0.1825  -2.1430
1      Topic1  2.157667     system  3.542785   0.1794  -1.9230
0      Topic1  1.713996      trees  2.834018   0.1724  -2.1532
6      Topic1  1.283804        eps  2.125484   0.1711  -2.4422
8      Topic1  1.075795      human  2.121877  -0.0039  -2.6189
5      Topic1  1.054314     survey  2.121505  -0.0239  -2.6391
9      Topic1  1.309957       user  2.827012  -0.0939  -2.4220
10     Topic1  0.840908  interface  2.117804  -0.2483  -2.8653
3      Topic1  0.731413   computer  2.115906  -0.3870  -3.0048
2      Topic1  0.677674   response  2.114974  -0.4628  -3.0811
11     Topic1  0.668090       time  2.114808  -0.4770  -3.0953
11     Topic2  1.446717       time  2.114808   0.3316  -2.2867
2      Topic2  1.437300   response  2.114974   0.3250  -2.2932
3      Topic2  1.384493   computer  2.115906   0.2872  -2.3307
10     Topic2  1.276896  interface  2.117804   0.2054  -2.4116
9      Topic2  1.517055       user  2.827012   0.0889  -2.2392
5      Topic2  1.067191     survey  2.121505   0.0242  -2.5910
8      Topic2  1.046082      human  2.121877   0.0041  -2.6109
6      Topic2  0.841679        eps  2.125484  -0.2150  -2.8283
0      Topic2  1.120022      trees  2.834018  -0.2170  -2.5426
1      Topic2  1.385118     system  3.542785  -0.2278  -2.3302
4      Topic2  1.102770      graph  2.834322  -0.2327  -2.5582
7      Topic2  0.613676     minors  2.129507  -0.5329  -3.1443, token_table=      Topic      Freq       Term
term                            
3         1  0.472611   computer
3         2  0.472611   computer
6         1  0.470481        eps
6         2  0.470481        eps
4         1  0.705636      graph
4         2  0.352818      graph
8         1  0.471281      human
8         2  0.471281      human
10        1  0.472187  interface
10        2  0.472187  interface
7         1  0.939184     minors
7         2  0.469592     minors
2         1  0.472819   response
2         2  0.472819   response
5         1  0.471364     survey
5         2  0.471364     survey
1         1  0.564528     system
1         2  0.282264     system
11        1  0.472856       time
11        2  0.472856       time
0         1  0.705712      trees
0         2  0.352856      trees
9         1  0.353730       user
9         2  0.707461       user, R=12, lambda_step=0.01, plot_opts={'ylab': 'PC2', 'xlab': 'PC1'}, topic_order=[1, 2])

In [11]:
pyLDAvis.gensim.prepare(badLdaModel, corpus, dictionary)

DEBUG:gensim.models.ldamodel:performing inference on a chunk of 9 documents
DEBUG:gensim.models.ldamodel:0/9 documents converged within 1 iterations


PreparedData(topic_coordinates=            Freq  cluster  topics         x    y
topic                                           
0      52.048726        1       1  0.001373  0.0
1      47.951274        1       2 -0.001373  0.0, topic_info=     Category      Freq       Term     Total  loglift  logprob
term                                                          
11    Default  2.000000       time  2.000000  12.0000  12.0000
6     Default  2.000000        eps  2.000000  11.0000  11.0000
0     Default  2.000000      trees  2.000000  10.0000  10.0000
9     Default  2.000000       user  2.000000   9.0000   9.0000
1     Default  3.000000     system  3.000000   8.0000   8.0000
3     Default  2.000000   computer  2.000000   7.0000   7.0000
5     Default  2.000000     survey  2.000000   6.0000   6.0000
10    Default  2.000000  interface  2.000000   5.0000   5.0000
8     Default  2.000000      human  2.000000   4.0000   4.0000
7     Default  2.000000     minors  2.000000   3.0000   3.0000
4     Default  2.000000      graph  2.000000   2.0000   2.0000
2     Default  2.000000   response  2.000000   1.0000   1.0000
9      Topic1  1.628865       user  2.832081   0.0999  -2.2264
10     Topic1  1.192640  interface  2.123539   0.0761  -2.5381
8      Topic1  1.174337      human  2.123209   0.0608  -2.5536
2      Topic1  1.149910   response  2.122770   0.0400  -2.5746
4      Topic1  1.521686      graph  2.830152   0.0325  -2.2945
7      Topic1  1.123072     minors  2.122286   0.0166  -2.5982
1      Topic1  1.822732     system  3.536261  -0.0097  -2.1140
5      Topic1  1.089144     survey  2.121676  -0.0138  -2.6289
3      Topic1  1.083093   computer  2.121567  -0.0193  -2.6345
0      Topic1  1.380666      trees  2.827613  -0.0639  -2.3917
6      Topic1  1.007627        eps  2.120208  -0.0909  -2.7067
11     Topic1  0.920360       time  2.118637  -0.1808  -2.7973
11     Topic2  1.198277       time  2.118637   0.1651  -2.4514
6      Topic2  1.112581        eps  2.120208   0.0902  -2.5256
0      Topic2  1.446948      trees  2.827613   0.0650  -2.2629
3      Topic2  1.038474   computer  2.121567   0.0206  -2.5946
5      Topic2  1.032532     survey  2.121676   0.0148  -2.6003
1      Topic2  1.713529     system  3.536261   0.0105  -2.0938
7      Topic2  0.999215     minors  2.122286  -0.0183  -2.6331
4      Topic2  1.308466      graph  2.830152  -0.0365  -2.3635
2      Topic2  0.972860   response  2.122770  -0.0453  -2.6598
8      Topic2  0.948873      human  2.123209  -0.0704  -2.6848
10     Topic2  0.930899  interface  2.123539  -0.0897  -2.7039
9      Topic2  1.203217       user  2.832081  -0.1210  -2.4473, token_table=      Topic      Freq       Term
term                            
3         1  0.471350   computer
3         2  0.471350   computer
6         1  0.471652        eps
6         2  0.471652        eps
4         1  0.706676      graph
4         2  0.353338      graph
8         1  0.470985      human
8         2  0.470985      human
10        1  0.470912  interface
10        2  0.470912  interface
7         1  0.471190     minors
7         2  0.471190     minors
2         1  0.471083   response
2         2  0.471083   response
5         1  0.471326     survey
5         2  0.471326     survey
1         1  0.565569     system
1         2  0.565569     system
11        1  0.472002       time
11        2  0.472002       time
0         1  0.353655      trees
0         2  0.353655      trees
9         1  0.706194       user
9         2  0.353097       user, R=12, lambda_step=0.01, plot_opts={'ylab': 'PC2', 'xlab': 'PC1'}, topic_order=[1, 2])

In [12]:
print(goodcm.get_coherence())

-13.8440816966


In [14]:
print(badcm.get_coherence())

-14.1608330118


### Using C_V coherence

In [15]:
goodcm = CoherenceModel(model=goodLdaModel, texts=texts, dictionary=dictionary, coherence='c_v')

In [16]:
badcm = CoherenceModel(model=badLdaModel, texts=texts, dictionary=dictionary, coherence='c_v')

### Pipeline parameters for C_V coherence

In [17]:
print(goodcm)

Coherence_Measure(seg=<function s_one_set at 0x7f90272c4a60>, prob=<function p_boolean_sliding_window at 0x7f90272c4bf8>, conf=<function cosine_similarity at 0x7f90272c4ea0>, aggr=<function arithmetic_mean at 0x7f90272c4f28>)


### Print coherence values

In [18]:
print(goodcm.get_coherence())

0.375262855326


In [19]:
print(badcm.get_coherence())

0.358036284922


### Support for wrappers

This API supports gensim's _ldavowpalwabbit_ and _ldamallet_ wrappers as input parameter to `model`.

In [25]:
try:
    model1 = LdaVowpalWabbit('/home/devashish/vw-8', corpus=corpus, num_topics=2, id2word=dictionary, passes=50)
    model2 = LdaVowpalWabbit('/home/devashish/vw-8', corpus=corpus, num_topics=2, id2word=dictionary, passes=1)
except FileNotFoundError:
    raise ValueError("SKIP: Please change the path")

INFO:gensim.models.wrappers.ldavowpalwabbit:using /tmp/tmp5c5c2ll7 as temp dir
DEBUG:gensim.models.wrappers.ldavowpalwabbit:Training new model from corpus
DEBUG:gensim.models.wrappers.ldavowpalwabbit:Writing corpus to: /tmp/tmp5c5c2ll7/corpus.vw
INFO:gensim.models.wrappers.ldavowpalwabbit:Running Vowpal Wabbit command: /home/devashish/vw-8 -d /tmp/tmp5c5c2ll7/corpus.vw --power_t 0.5 --initial_t 1 --minibatch 256 --lda_D 9 --passes 50 --cache_file /tmp/tmp5c5c2ll7/cache.vw --lda_epsilon 0.001 --readable_model /tmp/tmp5c5c2ll7/topics.vw -k -f /tmp/tmp5c5c2ll7/model.vw --lda 2 -b 4 --lda_alpha 0.1 --lda_rho 0.1


ValueError: SKIP: Please change the path

In [23]:
cm1 = CoherenceModel(model=model1, corpus=corpus, coherence='u_mass')
cm2 = CoherenceModel(model=model2, corpus=corpus, coherence='u_mass')

NameError: name 'model1' is not defined

In [22]:
print(cm1.get_coherence())
print(cm2.get_coherence())

NameError: name 'cm1' is not defined

In [20]:
model1 = LdaMallet('/home/devashish/mallet-2.0.8RC3/bin/mallet',corpus=corpus , num_topics=2, id2word=dictionary, iterations=50)
model2 = LdaMallet('/home/devashish/mallet-2.0.8RC3/bin/mallet',corpus=corpus , num_topics=2, id2word=dictionary, iterations=1)

In [21]:
cm1 = CoherenceModel(model=model1, texts=texts, coherence='c_v')
cm2 = CoherenceModel(model=model2, texts=texts, coherence='c_v')

In [22]:
print cm1.get_coherence()
print cm2.get_coherence()

0.581114877802
0.549865328265


### Support for other topic models
The gensim topics coherence pipeline can be used with other topics models too. Only the tokenized `topics` should be made available for the pipeline. Eg. with the gensim HDP model

In [5]:
hm = HdpModel(corpus=corpus, id2word=dictionary)

In [8]:
# To get the topic words from the model
topics = []
for topic_id, topic in hm.show_topics(num_topics=10, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

In [9]:
topics[:2]

[[u'minors',
  u'system',
  u'graph',
  u'human',
  u'interface',
  u'eps',
  u'trees',
  u'computer',
  u'user',
  u'response',
  u'survey',
  u'time'],
 [u'minors',
  u'trees',
  u'time',
  u'interface',
  u'user',
  u'survey',
  u'system',
  u'response',
  u'human',
  u'computer',
  u'graph',
  u'eps']]

In [10]:
# Initialize CoherenceModel using `topics` parameter
cm = CoherenceModel(topics=topics, corpus=corpus, dictionary=dictionary, coherence='u_mass')

In [11]:
cm.get_coherence()

-14.640667699204982

### Conclusion

Hence as we can see, the `u_mass` and `c_v` coherence for the good LDA model is much more (better) than that for the bad LDA model. This is because, simply, the good LDA model usually comes up with better topics that are more human interpretable. The badLdaModel however fails to decipher between these two topics and comes up with topics which are not clear to a human. The `u_mass` and `c_v` topic coherences capture this wonderfully by giving the interpretability of these topics a number as we can see above. Hence this coherence measure can be used to compare difference topic models based on their human-interpretability.